<a href="https://colab.research.google.com/github/ArpanSM/Machine_Learning_Hackathons/blob/master/Deep%20Learning%20Challenge%20Auto%20Tag%20Images%20Gala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten,Conv2D,BatchNormalization,Dropout
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#print(tf.__version__)

#GPU Training
import keras
import tensorflow as tf


config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 4} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)



training_set = pd.read_csv("./train.csv")
training_imgs = ["{}".format(x) for x in list(training_set.Image)]
training_label= list(training_set['Class'])
training_set = pd.DataFrame( {'Images': training_imgs,'Class': training_label} )
#Changing the type  to str
training_set.Class = training_set.Class.astype(str)
#print(training_set.head())

#data-preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dataGen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)
train_generator = train_dataGen.flow_from_dataframe(
                                        dataframe = training_set,
                                        directory="./Train",
                                        x_col="Images",
                                        y_col="Class",
                                        class_mode="categorical",
                                        target_size=(128,128),
                                        batch_size=32)

input_shape = (128,128,3)
num_classes = 4

#Building Convolutional Neural Network
classifier = Sequential()
#First Convolutional layer
classifier.add(Convolution2D(filters = 64,kernel_size = (3,3), activation = 'relu', input_shape = (128,128,3)))
classifier.add(MaxPooling2D(pool_size = (2,2)))
#second Convolutional layer
classifier.add(Convolution2D(128,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
#third
classifier.add(Convolution2D(256,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
#Flattening
classifier.add(Flatten())
#Hidden Layer
classifier.add(Dense(units = 64, activation = 'relu'))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 256, activation = 'relu'))
#Output Layer
classifier.add(Dense(units = 4 , activation = 'softmax'))
classifier.compile(optimizer = 'Adamax', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy','accuracy'])
#classifier.summary()
#Training the CNN
classifier.fit_generator(train_generator, epochs = 10, steps_per_epoch = 100 )



#Predicting For Test Set
test_set = pd.read_csv("C:/Users/arpan/OneDrive/Desktop/Gala_Classify/test.csv")
test_imgs = ["./Test/{}".format(x) for x in list(test_set.Image)]
test_set = pd.DataFrame( {'Images': test_imgs })
classes = train_generator.class_indices
print(classes)
inverted_classes = dict(map(reversed, classes.items()))
print(inverted_classes)


#Predicting classes
from keras.preprocessing import image
Y_pred = []
for i in range(len(test_set)):
  img = image.load_img(path= test_set.Images[i],target_size=(128,128,3))
  img = image.img_to_array(img)
  test_img = img.reshape((1,128,128,3))
  img_class = classifier.predict_classes(test_img)
  prediction = img_class[0]
  Y_pred.append(prediction)
#print(Y_pred)
prediction_classes = [ inverted_classes.get(item,item) for item in Y_pred ]
#print(prediction_classes)



classifier.save_weights("model.h5")
print("Saved model to disk")

#Writing to excel
#test_set['Class'] = prediction_classes
#test_set.to_csv('test.csv', index=False)